In [32]:
%%html
<style type='text/css'>
.CodeMirror{
    font-family: Inconsolata;
    font-size: 13px;
}

div.output_area pre {
    font-family: Inconsolata;
    font-size: 11px;
}

div.text_cell_render {
    font-family: Inconsolata;
    font-size: 13px;
}
</style>

In [33]:
import pandas as pd
import os, shutil
import numpy as np
import matplotlib
import sys; sys.path.append('..')
import pickle
import requests
import torch

from queue import Queue
from tqdm.notebook import tqdm, tqdm_notebook
from collections import defaultdict

path_ddi_dataset = r"/data/data2/041/datasets/DDI"
path_iii_dataset = r"/data/data2/041/datasets/mimic-iii-clinical-database-1.4"

# `process_med`

In [34]:
med_pd = pd.read_csv(os.path.join(path_iii_dataset, "PRESCRIPTIONS.csv.gz"), dtype={'NDC':'category'})
med_pd.info()

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4156450 entries, 0 to 4156449
Data columns (total 19 columns):
 #   Column             Dtype   
---  ------             -----   
 0   ROW_ID             int64   
 1   SUBJECT_ID         int64   
 2   HADM_ID            int64   
 3   ICUSTAY_ID         float64 
 4   STARTDATE          object  
 5   ENDDATE            object  
 6   DRUG_TYPE          object  
 7   DRUG               object  
 8   DRUG_NAME_POE      object  
 9   DRUG_NAME_GENERIC  object  
 10  FORMULARY_DRUG_CD  object  
 11  GSN                object  
 12  NDC                category
 13  PROD_STRENGTH      object  
 14  DOSE_VAL_RX        object  
 15  DOSE_UNIT_RX       object  
 16  FORM_VAL_DISP      object  
 17  FORM_UNIT_DISP     object  
 18  ROUTE              object  
dtypes: category(1), float64(1), int64(3), object(14)
memory usage: 578.9+ MB


In [35]:
med_pd.drop(columns=['ROW_ID','DRUG_TYPE','DRUG_NAME_POE','DRUG_NAME_GENERIC',
                 'FORMULARY_DRUG_CD','GSN','PROD_STRENGTH','DOSE_VAL_RX',
                 'DOSE_UNIT_RX','FORM_VAL_DISP','FORM_UNIT_DISP','FORM_UNIT_DISP',
                  'ROUTE','ENDDATE','DRUG'], axis=1, inplace=True)
med_pd.drop(index = med_pd[med_pd['NDC'] == '0'].index, axis=0, inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569864 entries, 0 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 143.1+ MB


In [36]:
med_pd.fillna(method='pad', inplace=True)
med_pd.dropna(inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569840 entries, 30 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 143.1+ MB


In [37]:
med_pd.drop_duplicates(inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2987420 entries, 30 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 119.8+ MB


In [38]:
med_pd['ICUSTAY_ID'] = med_pd['ICUSTAY_ID'].astype('int64')
med_pd['STARTDATE'] = pd.to_datetime(med_pd['STARTDATE'], format='%Y-%m-%d %H:%M:%S')    
med_pd.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE'], inplace=True)
med_pd = med_pd.reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2987420 entries, 0 to 2987419
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   HADM_ID     int64         
 2   ICUSTAY_ID  int64         
 3   STARTDATE   datetime64[ns]
 4   NDC         category      
dtypes: category(1), datetime64[ns](1), int64(3)
memory usage: 97.0 MB


In [39]:
def filter_first24hour_med(med_pd):
    med_pd_new = med_pd.drop(columns=['NDC'])
    med_pd_new = med_pd_new.groupby(by=['SUBJECT_ID','HADM_ID','ICUSTAY_ID']).head([1]).reset_index(drop=True)
    med_pd_new = pd.merge(med_pd_new, med_pd, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTDATE'])
    med_pd_new = med_pd_new.drop(columns=['STARTDATE'])
    return med_pd_new

med_pd = filter_first24hour_med(med_pd)
# med_pd = med_pd.drop(columns=['STARTDATE'])
med_pd = med_pd.drop(columns=['ICUSTAY_ID'])
med_pd = med_pd.drop_duplicates()
med_pd = med_pd.reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137655 entries, 0 to 1137654
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   SUBJECT_ID  1137655 non-null  int64   
 1   HADM_ID     1137655 non-null  int64   
 2   NDC         1137655 non-null  category
dtypes: category(1), int64(2)
memory usage: 19.7 MB


In [40]:
# visit > 2
def process_visit_lg2(med_pd):
    a = med_pd[['SUBJECT_ID', 'HADM_ID']].groupby(by='SUBJECT_ID')['HADM_ID'].unique().reset_index()
    a['HADM_ID_Len'] = a['HADM_ID'].map(lambda x:len(x))
    a = a[a['HADM_ID_Len'] > 1]
    return a 

med_pd_lg2 = process_visit_lg2(med_pd).reset_index(drop=True)
med_pd = med_pd.merge(med_pd_lg2[['SUBJECT_ID']], on='SUBJECT_ID', how='inner')
med_pd.reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420093 entries, 0 to 420092
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   SUBJECT_ID  420093 non-null  int64   
 1   HADM_ID     420093 non-null  int64   
 2   NDC         420093 non-null  category
dtypes: category(1), int64(2)
memory usage: 10.6 MB


In [41]:
len(med_pd.HADM_ID.unique()), len(med_pd.NDC.unique())

(17368, 3329)

# diagnoses

In [42]:
diag_file = os.path.join(path_iii_dataset, "DIAGNOSES_ICD.csv.gz")

def process_diag():
    diag_pd = pd.read_csv(diag_file)
    diag_pd.dropna(inplace=True)
    diag_pd.drop(columns=['SEQ_NUM','ROW_ID'],inplace=True)
    diag_pd.drop_duplicates(inplace=True)
    diag_pd.sort_values(by=['SUBJECT_ID','HADM_ID'], inplace=True)
    
    return diag_pd.reset_index(drop=True)

def filter_2000_most_diag(diag_pd):
    diag_count = diag_pd.groupby(by=['ICD9_CODE']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['count'],ascending=False).reset_index(drop=True)
    diag_pd = diag_pd[diag_pd['ICD9_CODE'].isin(diag_count.loc[:1999, 'ICD9_CODE'])]
    
    return diag_pd.reset_index(drop=True)

diag_pd = process_diag()
diag_pd = filter_2000_most_diag(diag_pd)
diag_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625434 entries, 0 to 625433
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   SUBJECT_ID  625434 non-null  int64 
 1   HADM_ID     625434 non-null  int64 
 2   ICD9_CODE   625434 non-null  object
dtypes: int64(2), object(1)
memory usage: 14.3+ MB


# procedures

In [43]:
procedure_file = os.path.join(path_iii_dataset, "PROCEDURES_ICD.csv.gz")

def process_procedure():
    pro_pd = pd.read_csv(procedure_file, dtype={'ICD9_CODE':'category'})
    pro_pd.drop(columns=['ROW_ID'], inplace=True)
    
    pro_pd.drop_duplicates(inplace=True)
    pro_pd.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM'], inplace=True)
    pro_pd.drop(columns=['SEQ_NUM'], inplace=True)
    pro_pd.drop_duplicates(inplace=True)
    pro_pd.reset_index(drop=True, inplace=True)
    
    return pro_pd

pro_pd = process_procedure()
pro_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228679 entries, 0 to 228678
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   SUBJECT_ID  228679 non-null  int64   
 1   HADM_ID     228679 non-null  int64   
 2   ICD9_CODE   228679 non-null  category
dtypes: category(1), int64(2)
memory usage: 4.0 MB


# `process_all`

In [44]:
med_pd_key = med_pd[['SUBJECT_ID', 'HADM_ID']].drop_duplicates()
diag_pd_key = diag_pd[['SUBJECT_ID', 'HADM_ID']].drop_duplicates()
pro_pd_key = pro_pd[['SUBJECT_ID', 'HADM_ID']].drop_duplicates()

combined_key = med_pd_key.merge(diag_pd_key, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
combined_key = combined_key.merge(pro_pd_key, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [45]:
diag_pd = diag_pd.merge(combined_key, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
med_pd = med_pd.merge(combined_key, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
pro_pd = pro_pd.merge(combined_key, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374466 entries, 0 to 374465
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   SUBJECT_ID  374466 non-null  int64   
 1   HADM_ID     374466 non-null  int64   
 2   NDC         374466 non-null  category
dtypes: category(1), int64(2)
memory usage: 9.4 MB


In [46]:
# flatten and merge
diag_pd = diag_pd.groupby(by=['SUBJECT_ID','HADM_ID'])['ICD9_CODE'].unique().reset_index()  
med_pd = med_pd.groupby(by=['SUBJECT_ID', 'HADM_ID'])['NDC'].unique().reset_index()
pro_pd = pro_pd.groupby(by=['SUBJECT_ID','HADM_ID'])['ICD9_CODE'].unique().reset_index().rename(columns={'ICD9_CODE':'PRO_CODE'})  

med_pd['NDC'] = med_pd['NDC'].map(lambda x: list(x))
pro_pd['PRO_CODE'] = pro_pd['PRO_CODE'].map(lambda x: list(x))

data = diag_pd.merge(med_pd, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
data = data.merge(pro_pd, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

data['NDC_Len'] = data['NDC'].map(lambda x: len(x))
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15041 entries, 0 to 15040
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SUBJECT_ID  15041 non-null  int64 
 1   HADM_ID     15041 non-null  int64 
 2   ICD9_CODE   15041 non-null  object
 3   NDC         15041 non-null  object
 4   PRO_CODE    15041 non-null  object
 5   NDC_Len     15041 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 822.6+ KB


In [47]:
data.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE,NDC,PRO_CODE,NDC_Len
0,17,161087,"[4239, 5119, 78551, 4589, 311, 7220, 71946, 2724]","[00713016550, 00904770418, 00904404073, 009045...","[3731, 8872, 3893]",36
1,17,194023,"[7455, 45829, V1259, 2724]","[51079087101, 55390048101, 00338011704, 000741...","[3571, 3961, 8872]",32
2,21,109451,"[41071, 78551, 5781, 5849, 40391, 4280, 4592, ...","[55390007310, 66553000401, 00045152010, 510790...","[0066, 3761, 3950, 3606, 0042, 0047, 3895, 399...",31
3,21,111970,"[0388, 78552, 40391, 42731, 70709, 5119, 6823,...","[00338055318, 00074455211, 00310032520, 003380...","[3995, 8961, 0014]",26
4,23,124321,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[00071036940, 00006494300, 00054817525, 004091...",[0151],14


In [48]:
len(data.HADM_ID.unique())

15041